<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **スペース X ファルコン 9 第一段着陸予測**


 ## Lab 2: Data wrangling 


Estimated time needed: **60** minutes


このラボでは、探索的データ分析 (EDA) を実行して、データ内のいくつかのパターンを見つけ、教師ありモデルをトレーニングするためのラベルを決定します。
データセットには、ブースターが正常に着陸しなかったいくつかの異なるケースがあります。時々、着陸が試みられたが、事故のために失敗した。たとえば、<code>True Ocean</code> は、ミッションの結果が海洋の特定の領域に正常に着陸したことを意味し、<code>False Ocean</code> は、ミッションの結果が海洋の特定の領域に正常に着陸できなかったことを意味します。 。 <code>True RTLS</code> は、ミッションの結果が地上パッドへの着陸に成功したことを意味します。<code>False RTLS</code> は、ミッションの結果が地上パッドへの着陸に失敗したことを意味します。<code>True ASDS</code> <code>False ASDS</code> は、ミッションの結果がドローン船への着陸に失敗したことを意味します。
このラボでは、主にこれらの結果をトレーニング ラベルに変換します。「1」はブースターが着地に成功したことを意味します。「0」はブースターが失敗したことを意味します。


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


## 目的
探索的なデータ分析を実行し、トレーニング ラベルを決定する 

-探索的データ分析
-トレーニングラベルの決定


----


Install the below libraries


In [ ]:
!pip install pandas
!pip install numpy

## Import Libraries and Define Auxiliary Functions


We will import the following libraries.


In [1]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

### Data Analysis 


Load Space X dataset, from last section.


In [2]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


各属性の欠損値の割合を特定して計算します。

In [3]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

どの列が数値列とカテゴリ列であるかを特定します。


In [4]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### タスク 1: 各サイトでの起動数を計算する

データには、Space X のいくつかの打ち上げ施設が含まれています: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites'>ケープカナベラル宇宙</a> 打ち上げ施設 40 <b>VAFB SLC 4E </b> 、ヴァンデンバーグ空軍基地宇宙打ち上げ施設 4E <b>(SLC-4E)</b>、ケネディ宇宙センター打ち上げ複合体 39A <b>KSC LC 39A </b>。各打ち上げの場所は列 <code>LaunchSite</code> に配置されます。


次にサイトごとの起動数を見てみましょう。

<code>LaunchSite</code> 列のメソッド <code>value_counts()</code> を使用して、各サイトの起動数を決定します。


In [5]:
# Apply value_counts() on column LaunchSite

df['LaunchSite'].value_counts()


LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64

Each launch aims to an dedicated orbit, and here are some common orbit types:




*<b>LEO</b>: 地球低軌道 (LEO) は、高度 2,000 km (1,200 マイル) 以下 (地球の半径の約 3 分の 1) の地球中心の軌道です。<br>[1]、または1 日あたり少なくとも 11.25 周期 (公転周期は 128 分以下)、離心率は 0.25 未満です。<br>[2]宇宙空間にある人工物体のほとんどは、LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a> にあります。
*<b>VLEO</b>: 超低軌道 (VLEO) は、平均高度が 450 km 未満の軌道として定義できます。これらの軌道で運用することは、地球観測探査機が観測に近づいて運用するため、地球観測探査機に多くのメリットをもたらす可能性があります<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>。
*<b>GTO</b> 静止軌道は、衛星が地球の自転と一致することを可能にする地球の高軌道です。地球の赤道上空 22,236 マイル (35,786 キロメートル) に位置するこの位置は、気象、通信、監視を監視す​​るための貴重な場所です。衛星は地球の回転と同じ速度で周回しているため、衛星は北から南に流れる可能性はあるものの、単一の経度にわたってその位置に留まっているように見えます」と NASA は地球観測所の Web サイト <a href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>。
*<b>SSO (または SO)</b>: 太陽同期軌道であり、太陽同期軌道とも呼ばれます。これは、衛星が惑星の表面の任意の点の上を通過する、惑星の周りのほぼ極軌道です。地方平均太陽時と同じ <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>。
*<b>ES-L1 </b>: ラグランジュ点では、軌道上に置かれた小さな物体が大きな天体の質量中心に対して平衡になるように、2 つの大きな天体の重力が相殺されます。身体。 L1 は、太陽と地球の間にある点の 1 つです <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> 。
*<b>HEO</b> 高度な楕円軌道とは、離心率の高い楕円軌道で、通常は地球の周りの軌道を指します <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6] ]</a>。
*<b> ISS </b> 地球低軌道にあるモジュール式宇宙ステーション (居住可能な人工衛星)。これは、NASA (米国)、ロスコスモス (ロシア)、JAXA (日本)、ESA (ヨーロッパ)、CSA (カナダ) の 5 つの参加宇宙機関による多国間共同プロジェクトです<a href="https://en.wikipedia .org/wiki/International_Space_Station"> [7] </a>
*<b> MEO </b> 高度 2,000 km (1,200 マイル) から静止軌道の直下 35,786 キロ (22,236 マイル) までの範囲の地心軌道。中間円軌道とも呼ばれます。これらは「最も一般的には高度 20,200 キロメートル (12,600 マイル) または 20,650 キロメートル (12,830 マイル) で、公転周期は 12 時間です<a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] ]</a>
*<b> HEO </b> 静止軌道高度 (35,786 km または 22,236 マイル) を超える地心軌道 >


*<b> GEO </b> 地球の赤道上空 35,786 キロメートル (22,236 マイル) にあり、地球の自転方向に沿った円形の静止軌道です。 > [10] </a>
*<b> PO </b> 衛星の一種で、衛星が周回している天体の両極の上またはほぼ上を通過します (通常は地球などの惑星 <a href="https://en. wikipedia.org/wiki/Polar_orbit"> [11] </a>

一部を次のプロットに示します。


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)


### タスク 2: 各軌道の数と発生を計算する


メソッド <code>.value_counts()</code> を使用して、列 <code>Orbit</code> 内の各軌道の数と発生を確認します。


In [6]:
# Apply value_counts on Orbit column

df['Orbit'].value_counts()

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: count, dtype: int64

### タスク 3: 軌道のミッション結果の数と発生を計算する

<code>Outcome</code> 列のメソッド <code>.value_counts()</code> を使用して、<code>landing_outcomes</code> の数を決定します。それを変数 landing_outcomes に割り当てます。


In [9]:
# landing_outcomes = values on Outcome column

landing_outcomes = df['Outcome'].value_counts()

landing_outcomes

Outcome
True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
False Ocean     2
None ASDS       2
False RTLS      1
Name: count, dtype: int64


<code>True Ocean</code> は、ミッションの結果が海洋の特定の領域に正常に着陸したことを意味し、<code>False Ocean</code> は、ミッションの結果が海洋の特定の領域に正常に着陸できなかったことを意味します。 <code>True RTLS</code> は、ミッションの結果が地上パッドへの着陸に成功したことを意味します。<code>False RTLS</code> は、ミッションの結果が地上パッドへの着陸に失敗したことを意味します。<code>True ASDS</code> <code>False ASDS</code> は、ミッションの結果がドローン船への着陸に失敗したことを意味します。 <code>なし ASDS</code> と <code>なし なし</code> は、着陸の失敗を表します。


In [10]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


第 2 ステージが正常に完了しなかった一連の結果を作成します。


In [11]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

### タスク 4: 結果列からランディング結果ラベルを作成する

<code>Outcome</code> を使用して、<code>Outcome</code> の対応する行がセット <code>bad_outcome</code> にある場合は要素が 0 であるリストを作成します。それ以外の場合は 1 つです。次に、それを変数 <code>landing_class</code> に割り当てます。


In [16]:
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwis

#リスト内包表記
# landing_class = [0 if outcome in bad_outcomes else 1 for outcome in df['Outcome']]

#forループ分
landing_class = []
for outcome in df['Outcome']:
    if outcome in bad_outcomes:
        landing_class.append(0)
    else:
        landing_class.append(1)

landing_class

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [22]:
# words = ['apple', 'banana', 'cherry']
# ng_words = ['apple']
# first_letters = [0 if word in ng_words else 1 for word in words]

# first_letters


This variable will represent the classification variable that represents the outcome of each launch. If the value is zero, the  first stage did not land successfully; one means  the first stage landed Successfully 


In [23]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [24]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


次のコード行を使用して、成功率を決定できます。


In [25]:
df["Class"].mean()

0.6666666666666666

In [27]:
df['Class'].value_counts()

Class
1    60
0    30
Name: count, dtype: int64

次のセクションでそれを CSV にエクスポートできますが、答えに一貫性を持たせるために、次のラボでは、事前に選択した日付範囲のデータを提供します。


<code>df.to_csv("dataset_part_2.csv", index=False)</code>


In [26]:
df.to_csv("dataset_part_2.csv", index=False)

## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a> is a Data Scientist at IBM and pursuing a Master of Management in Artificial intelligence degree at Queen's University.


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-08-31        | 1.1     | Lakshmi Holla    | Changed Markdown |
| 2020-09-20        | 1.0     | Joseph     | Modified Multiple Areas |
| 2020-11-04        | 1.1.    | Nayef      | updating the input data |
| 2021-05-026       | 1.1.    | Joseph      | updating the input data |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
